In [ ]:
!pip install  openai langchain sentence_transformers chromadb unstructured -q


In [ ]:
pip install -U langchain-community

In [ ]:
from langchain.document_loaders import DirectoryLoader

directory = '/content/sample_data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)


4

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))


4


In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)


In [ ]:
query = "What are the different kinds of pets people commonly own?"
matching_docs = db.similarity_search(query)

matching_docs[0]


Document(metadata={'source': '/content/sample_data/types of pets.txt'}, page_content='There are various types of pet animals, each offering unique companionship and joy to their owners. Dogs are renowned for their loyalty and playful nature, often referred to as "man\'s best friend." Cats, with their independent yet affectionate demeanor, are cherished for their elegance and purring companionship. Birds, ranging from colorful parrots to melodious canaries, bring a lively and musical atmosphere to homes. Small mammals like hamsters, guinea pigs, and rabbits are adored for their cuddliness and relatively low-maintenance care. Fish, with their vibrant colors and tranquil presence, provide a soothing and visually captivating experience. Reptiles, including turtles, lizards, and snakes, appeal to those fascinated by their unique behaviors and exotic charm. Each type of pet offers different experiences and requires specific care, making the choice of a pet a personal and rewarding decision.'

In [ ]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [40]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-sLPY2a1lpoSfmuSb7Bp1T3BlbkFJCwqmViv3uy0wCn2jJpUZ"

from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)


In [41]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = "What are the emotional benefits of owning a pet?"
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Pets are beloved companions that bring joy, comfort, and a sense of responsibility to their owners. They come in various forms, from dogs and cats to birds, fish, and even exotic animals. Pets provide emotional support, reduce stress, and encourage a more active lifestyle through activities like walking or playing. The bond between pets and their owners can be incredibly strong, offering unconditional love and companionship that enriches daily life.

Pets are beloved companions that bring joy, comfort, and a sense of responsibility to their owners. They come in various forms, from dogs and cats to birds, fish, and even exotic animals. Pets provide emotional support, reduce stress, and encourage a mor

'Owning a pet can bring various emotional benefits, such as providing companionship, reducing stress and anxiety, offering unconditional love, and promoting a sense of purpose and responsibility. Pets can be great listeners, offering a non-judgmental ear for their owners to express their thoughts and feelings. The bond between pets and their owners can help reduce feelings of loneliness and provide comfort during challenging times. Additionally, the playful and affectionate nature of pets can boost mood and overall well-being.'

In [42]:
from langchain.chains import RetrievalQA
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)


'Owning a pet can bring various emotional benefits to individuals. Pets provide companionship, which can help reduce feelings of loneliness and isolation. The bond between a pet and their owner can offer unconditional love, comfort, and support, leading to increased feelings of happiness and well-being. Interacting with pets has been shown to reduce stress levels and anxiety, promoting relaxation and a sense of calm. Additionally, caring for a pet can provide a sense of purpose, responsibility, and routine, which can be beneficial for mental health. Overall, owning a pet can enhance emotional well-being by fostering feelings of joy, connection, and fulfillment.'